In [1]:
import json
import os
import re
import argparse
import datetime
import google.generativeai as genai

# ─── Configuration ──────────────────────────────────────────────────────────────

def configure_gemini(api_key):
    """Configure the Gemini API with the provided API key."""
    genai.configure(api_key=api_key)
    return genai.GenerativeModel(
        "gemini-2.0-flash", 
        generation_config={"temperature": 0.3}  # Higher temperature for creative modifications
    )
# ─── File Reading Functions ────────────────────────────────────────────────────

def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return None


def read_modifications_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return None

# ─── Display & Modification ────────────────────────────────────────────────────

def display_modifications(modifications_text):
    if not modifications_text:
        print("No modifications available.")
        return False
    # print("\n=== AVAILABLE MODIFICATIONS ===\n")
    print(modifications_text)
    return True


def get_user_choice():
    print("\n What change would you like to make to your game?")
    return input("> ").strip()


def implement_modification(model, game_code, modification_request):
    html = game_code.get("html", "")
    css = game_code.get("css", "")
    js = game_code.get("javascript", "")
    prompt = f"""
You are an expert web developer modifying a Tic Tac Toe Plus game.
The user has requested the following modification:

"{modification_request}"

I need you to implement this modification and return the modified code.

IMPORTANT: Format your response EXACTLY as a valid JSON object with these three keys:
"html", "css", and "javascript".
Only make changes necessary for the requested modification.

Current HTML code:
{html}

Current CSS code:
{css}

Current JavaScript code:
{js}
"""
    response = model.generate_content(prompt)
    resp_text = response.text.strip()
    # Extract JSON and return a dict or fallback to originals
    match = re.search(r'({[\s\S]*})', resp_text)
    if match:
        try:
            data = json.loads(match.group(1))
            if all(k in data for k in ("html","css","javascript")):
                return data
        except json.JSONDecodeError:
            pass
    # Fallback
    return {"html": html, "css": css, "javascript": js}

# ─── Save & Compare ────────────────────────────────────────────────────────────

def compare_and_report_changes(original_code, modified_code):
    changes = False
    if original_code["html"] != modified_code["html"]:
        print("✓ HTML changes detected")
        changes = True
    else:
        print("✗ No HTML changes detected")
    if original_code["css"] != modified_code["css"]:
        print("✓ CSS changes detected")
        changes = True
    else:
        print("✗ No CSS changes detected")
    if original_code["javascript"] != modified_code["javascript"]:
        print("✓ JavaScript changes detected")
        changes = True
    else:
        print("✗ No JavaScript changes detected")
    return changes

# ─── Load Game Code ────────────────────────────────────────────────────────────

def load_game_code():
    game_code = read_json_file("game_code.json")
    if not game_code:
        try:
            with open("index.html","r",encoding="utf-8") as f:
                html = f.read()
            with open("style.css","r",encoding="utf-8") as f:
                css = f.read()
            with open("script.js","r",encoding="utf-8") as f:
                js = f.read()
            game_code = {"html": html, "css": css, "javascript": js}
            with open("game_code.json","w",encoding="utf-8") as f:
                json.dump(game_code,f,indent=2)
            print("Game code loaded and saved to game_code.json")
        except FileNotFoundError as e:
            print(f"Error loading game files: {e}")
            return None
    return game_code


def select_game():
    """Prompt user; only Tic Tac Toe Plus is supported."""
    choice  = input(
        "What game would you like to build today?\n"
        "1. Tic Tac Toe\n" 
        # "2. Dots and Boxes\n"
        # "3. Checkers\n"
        # "4. Rock, Paper, Scissors\n"
        # "5. Memory Matching Game\n"
        # "6. Connect Four\n"
        # "7. Four in a Row\n"
        # "8. Battleship\n"
        # "9. Hangman\n"
        # "10. Othello\n"
        "Enter your choice: "
    ).strip().lower()

    if choice in ("1", "tic tac toe", "tic tac toe plus"):
        print("✅ Selected Tic Tac Toe Plus")
        return "Tic Tac Toe Plus"
    print(" That game does not exist. Please try out another idea.")
    return None
    
def check_request_clarity(model, modification_request):
    """Check if the modification request is clear enough to implement."""
    prompt = f"""
You are evaluating whether a user's request for modifying a Tic Tac Toe Plus game is clear enough.
The user has requested: "{modification_request}"

Is this request specific and clear enough to implement without additional information?
If YES, respond with only: "CLEAR: [brief explanation why it's clear]"
If NO, respond with only: "UNCLEAR: [possible interpretations separated by 'OR']"

For example:
- If request is "add a timer", respond with "UNCLEAR: Did you mean add a countdown timer for each move? OR add a game clock for the entire match? OR add a timer for something else?"
- If request is "make the X and O pieces red and blue", respond with "CLEAR: Colors for X and O are specifically defined as red and blue"
"""
    response = model.generate_content(prompt)
    resp_text = response.text.strip()
    
    if resp_text.startswith("CLEAR:"):
        return True, None
    elif resp_text.startswith("UNCLEAR:"):
        return False, resp_text[8:].strip()  # Remove "UNCLEAR: " prefix
    else:
        # Fallback in case the model doesn't follow the format
        return True, None

def check_request_scope(model, modification_request):
    """Check if the modification request is within scope of acceptable changes."""
    prompt = f"""
You are evaluating whether a user's request for modifying a Tic Tac Toe Plus game is within acceptable scope.

The user has requested: "{modification_request}"

In-scope changes (acceptable):
1. Add an AI opponent – implement simple random-move bot or Minimax algorithm
2. Mobile-first / responsive layout – make grid resize for phones/tablets
3. Theme & skin system – colors, fonts, emoji pieces, dark mode, holiday themes
4. Sound FX & animations – play sounds, shake invalid moves, animate winning line
5. Highlight winning combo – flash or outline winning squares
6. Scoreboard & match series – track wins/losses/draws with reset button
7. Turn timer – countdown per move or per game
8. Undo / redo – allow step back or full move history navigation
9. Keyboard shortcuts & accessibility – arrow keys, ARIA roles, screen-reader labels
10. Persist state – save board/scores to localStorage
11. Player name input & avatars – prompt for names and display them
12. Remote play (same rules) – use WebSockets/Firebase for real-time 3×3 grid
13. Statistics panel – win rate, game length, first-move advantage stats
14. Tutorial & hint system – teach strategy and show recommended moves

Out-of-scope changes (NOT acceptable):
1. Switching to Connect 4, Gomoku, 3-D Tic-Tac-Toe, or larger boards
2. Changing the win condition (different patterns or more than 3-in-a-row)
3. Adding more than two players or simultaneous turns
4. Hex-grid or circular boards that alter adjacency rules
5. Turning the project into Hangman, Snake, Chess, or any unrelated game
6. Power-ups or special abilities that break the X/O placement mechanic
7. Real-money wagers or micro-transactions
8. Augmented-reality or 3-D visualizations that replace the 2-D grid

Is this request within scope or out of scope?
If IN SCOPE, respond with only: "IN_SCOPE: [brief explanation why it's in scope]"
If OUT OF SCOPE, respond with only: "OUT_OF_SCOPE: [brief explanation why it's out of scope]"
"""
    response = model.generate_content(prompt)
    resp_text = response.text.strip()
    
    if resp_text.startswith("IN_SCOPE:"):
        return True, resp_text[9:].strip()  # Remove "IN_SCOPE: " prefix
    elif resp_text.startswith("OUT_OF_SCOPE:"):
        return False, resp_text[12:].strip()  # Remove "OUT_OF_SCOPE: " prefix
    else:
        # Fallback in case the model doesn't follow the format
        return True, "Unable to determine scope, proceeding with caution."


# ─── Main Function ────────────────────────────────────────────────────────────
def main():
    try:
        parser = argparse.ArgumentParser(description="Build or modify a Tic Tac Toe Plus game")
        parser.add_argument('--new', action='store_true', help='Generate a new game from scratch')
        args = parser.parse_known_args()[0]
        api_key = os.getenv("GEMINI_API_KEY", "YOUR_API_KEY")  ## ENTER YOUR API KEY HERE --------------------------------------------------------
        model = configure_gemini(api_key)
        
        # Game selection
        game = select_game()
        if not game:
            return
            
        new_game = args.new or True
        game_code = load_game_code()
        if not game_code: return
        
        print("Loading available modifications...")
        modifications_text = read_modifications_file("modifications.md")
        
        # Loop for implementing modifications until user types exit
        while True:
            if not display_modifications(modifications_text):
                break
                
            modification_request = get_user_choice()
            if not modification_request or modification_request.lower() == 'exit':
                print(" Awesome! You can see the applied changes to your game in the IDE.")
                break
            
            # Check if the request is clear
            is_clear, clarification_options = check_request_clarity(model, modification_request)
            
            if not is_clear:
                print(f"\nLet me clarify that request. {clarification_options}")
                print("\nPlease provide a clearer modification request.")
                modification_request = get_user_choice()
                if not modification_request or modification_request.lower() == 'exit':
                    print(" Awesome! You can see the applied changes to your game in the IDE.")
                    break
            
            # Check if the request is within scope
            is_in_scope, scope_explanation = check_request_scope(model, modification_request)
            
            if not is_in_scope:
                print(f"\nSorry, that modification is out of scope for this project: {scope_explanation}")
                print("\nPlease try a different modification that's within the project scope.")
                continue
            
            print(f"\nImplementing: {modification_request}")
            modified_code = implement_modification(model, game_code, modification_request)
            
            if not modified_code:
                print("Unfortunately, that's outside the scope of this project. Let's try something else instead.")
                continue
                
            print("\nAnalyzing changes...")
            changes_made = compare_and_report_changes(game_code, modified_code)
            
            # Save the modified code to files
            if isinstance(modified_code, dict):
                for file_type, content in modified_code.items():
                    filename = ""
                    if file_type == "html":
                        filename = "index.html"
                    elif file_type == "css":
                        filename = "style.css"
                    elif file_type == "javascript":
                        filename = "script.js"
                    
                    if filename:
                        with open(filename, 'w', encoding='utf-8') as f:
                            f.write(content)
                        print(f"Updated {filename}")
            else:
                # If it's a string, assume it's HTML
                with open('index.html', 'w', encoding='utf-8') as f:
                    f.write(modified_code)
                print("Updated index.html")
            
            # Update game_code for next modification
            game_code = modified_code
            
    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    main()

C:\Users\Varad Gorantyal\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


What game would you like to build today?
1. Tic Tac Toe
Enter your choice:  1


✅ Selected Tic Tac Toe Plus
Game code loaded and saved to game_code.json
Loading available modifications...
### In Scope Changes

_(Enhance or customize the classic 3 × 3, two-player Tic-Tac-Toe while keeping the core rules intact)_

1. **Add an AI opponent** – implement a simple random-move bot, then upgrade to the unbeatable Minimax algorithm with selectable difficulty.
2. **Mobile-first / responsive layout** – make the grid resize gracefully on phones and tablets and support touch events.
3. **Theme & skin system** – let players pick colors, fonts, emoji pieces, dark mode, or holiday themes.
4. **Sound FX & animations** – play click / win / draw sounds, shake invalid moves, and animate the winning line.
5. **Highlight winning combo** – flash or outline the three squares that complete a victory.
6. **Scoreboard & match series** – track wins, losses, draws, and best-of-N series with a reset button.
7. **Turn timer** – add a countdown per move or per game to encourage quick thinking.
8

>  exit


 Awesome! You can see the applied changes to your game in the IDE.
